# Prompt Engineering

In [1]:
import json

with open("../data/questions/train_sampled_questions_50000.json") as f:
   questions = list(json.load(f).items())

## Data Extraction

In [2]:
import random 

def compute_object_size(scene_graph, object):
    image_size = scene_graph["width"] * scene_graph["height"]
    object_size = object["w"] * object["h"]
    return object_size / image_size

class_samples_positive = []
attr_samples_positive = []
rel_samples_positive = []

def object_within_image_bounds(scene_graph, object):
    return object["x"] >= 0 and object["y"] >= 0 and object["h"] > 0 and object["w"] > 0 and \
           object["x"] + object["w"] <= scene_graph["width"] and object["y"] + object["h"] <= scene_graph["height"]

for qid, question in questions:
    for op in question["semantic"]:
        operation = op["operation"]
        argument = op["argument"].strip()
        objects = question["sceneGraph"]["objects"]

        if operation == "select" and argument != "scene" and not argument.endswith("(-)"):
            matching_objects = [(oid, objects[oid]) for oid in argument.split("(")[1][:-1].split(",") if object_within_image_bounds(question["sceneGraph"], objects[oid])] 
            if len(matching_objects) > 0:
                oid, object = random.choice(matching_objects)
                object["object_id"] = oid
                class_samples_positive.append({
                        "question_id": qid,
                        "question": question,
                        "image_id": question["imageId"],
                        "class": argument.split("(")[0].strip(),
                        "object": object,
                        "object_size": compute_object_size(question["sceneGraph"], object),
                        "y": True
                    })  

        elif operation.startswith("filter"):
            attr = ' '.join(operation.split(' ')[1:]) if operation != "filter" else "any"
            attr_value = argument[4:-1] if argument.startswith('not(') else argument
            objects_with_attr = [(oid, o) for oid, o in objects.items() if attr_value in o["attributes"] and object_within_image_bounds(question["sceneGraph"],o)]
            if len(objects_with_attr) > 0 and attr not in ["hposition", "vposition"]:
                oid, object = random.choice(objects_with_attr)
                object["object_id"] = oid
                attr_samples_positive.append({
                        "question_id": qid,
                        "question": question,
                        "image_id": question["imageId"],
                        "attr_value": attr_value,
                        "object": object,
                        "object_size": compute_object_size(question["sceneGraph"], object),
                        "y": True
                    })

        elif operation.startswith("verify"):
            attr = ' '.join(operation.split(' ')[1:]) if operation != "verify" else "any"
            attr_value = argument
            objects_with_attr = [(oid, o) for oid, o in objects.items() if attr_value in o["attributes"] and object_within_image_bounds(question["sceneGraph"], o)]
            if len(objects_with_attr) > 0 and attr not in ["hposition", "vposition"]:
                oid, object = random.choice(objects_with_attr)
                object["object_id"] = oid
                attr_samples_positive.append({
                        "question_id": qid,
                        "question": question,
                        "image_id": question["imageId"],
                        "attr_value": attr_value,
                        "object": object,
                        "object_size": compute_object_size(question["sceneGraph"], object),
                        "y": True
                    })
                
        elif operation.startswith("choose ") and argument != "":
            attr = " ".join(operation.split(" ")[1:])
            attr_value = random.choice([argument.split("|")[0], argument.split("|")[1]])
            objects_with_attr = [(oid, o) for oid, o in objects.items() if attr_value in o["attributes"] and object_within_image_bounds(question["sceneGraph"], o)]
            if len(objects_with_attr) > 0 and attr not in ["hposition", "vposition"]:
                oid, object = random.choice(objects_with_attr)
                object["object_id"] = oid
                attr_samples_positive.append({
                        "question_id": qid,
                        "question": question,
                        "image_id": question["imageId"],
                        "attr_value": attr_value,
                        "object": object,
                        "object_size": compute_object_size(question["sceneGraph"], object),
                        "y": True
                    })
                
        elif operation == "relate":
            relation_type = argument.split(',')[1]
            position = 'subject' if argument.split(',')[2].startswith('s') else 'object'
            target_object = argument.split('(')[1][:-1]

            if target_object != "-":
                if position == 'object':
                    matching_objects = [(oid, o) for oid, o in objects.items() if any(r["object"] == target_object and r["name"] == relation_type for r in o["relations"]) and object_within_image_bounds(question["sceneGraph"], o)]
                    if len(matching_objects) > 0:
                        oid0, object0 = random.choice(matching_objects)
                        object0["object_id"] = oid0
                        object1 = objects[target_object]
                        object1["object_id"] = target_object

                        rel_samples_positive.append({
                                "question_id": qid,
                                "question": question,
                                "image_id": question["imageId"],
                                "rel": relation_type,
                                "object0": object0,
                                "object1": object1,
                                "object0_size": compute_object_size(question["sceneGraph"], object0),
                                "object1_size": compute_object_size(question["sceneGraph"], object1),
                                "y": True
                            })
                        
                else:
                    matching_oids = [r["object"] for r in objects[target_object]["relations"] if r["name"] == relation_type]
                    matching_objects = [(oid, objects[oid]) for oid in matching_oids if object_within_image_bounds(question["sceneGraph"], objects[oid])]
                    if len(matching_objects) > 0:
                        object0 = objects[target_object]
                        object0["object_id"] = target_object
                        oid1, object1 = random.choice(matching_objects)
                        object1["object_id"] = oid1

                        rel_samples_positive.append({
                            "question_id": qid,
                            "question": question,
                            "image_id": question["imageId"],
                            "rel": relation_type,
                            "object0": object0,
                            "object1": object1,
                            "object0_size": compute_object_size(question["sceneGraph"], object0),
                            "object1_size": compute_object_size(question["sceneGraph"], object1),
                            "y": True
                        })

In [3]:
with open('../data/metadata/gqa_all_class.json') as f:
    categories = json.load(f)
class_to_category = {}
for category, classes in categories.items():
    for c in classes:
        if c not in class_to_category:
            class_to_category[c] = [category]
        else:
            class_to_category[c].append(category)

class_samples_negative = []
for sample in class_samples_positive:
    candidate = random.choice(class_samples_positive)
    while sample["class"] in [*class_to_category.get(candidate["object"]["name"], []), candidate["object"]["name"]]:
        candidate = random.choice(class_samples_positive)
    class_samples_negative.append({
        **candidate,
        "class": sample["class"],
        "y": False
    })
class_samples = [*class_samples_positive, *class_samples_negative]
random.shuffle(class_samples)

attr_samples_negative = []
for sample in attr_samples_positive:
    candidate = random.choice(attr_samples_positive)
    while sample["attr_value"] in candidate["object"]["attributes"]:
        candidate = random.choice(attr_samples_positive)
    
    attr_samples_negative.append({
        **candidate,
        "attr": sample["attr_value"],
        "y": False
    })
attr_samples = [*attr_samples_positive, *attr_samples_negative]
random.shuffle(attr_samples)

rel_samples_negative = []
for sample in rel_samples_positive:
    candidate = random.choice(rel_samples_positive)
    while any(r for r in candidate["object0"]["relations"] if r["name"] == sample["rel"] and r["object"] == candidate["object1"]["object_id"]):
        candidate = random.choice(rel_samples_positive)
    
    rel_samples_negative.append({
        **candidate,
        "rel": sample["rel"],
        "y": False
    })
rel_samples = [*rel_samples_positive, *rel_samples_negative]
random.shuffle(rel_samples)

## Testing

In [4]:
from transformers import CLIPProcessor, CLIPImageProcessor, CLIPTokenizer, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("mps")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
import tensorflow as tf
import pandas as pd

class_samples_flat = [{
    "question_id": s["question_id"],
    "image_id": s["image_id"],
    "bbox_x": s["object"]["x"],
    "bbox_y": s["object"]["y"],
    "bbox_w": s["object"]["w"],
    "bbox_h": s["object"]["h"],
    "bbox_size": s["object_size"],
    "class": s["class"],
    "y": s["y"]
} for s in class_samples]
class_samples_df = pd.DataFrame.from_dict(class_samples_flat)

attr_samples_flat = [{
    "question_id": s["question_id"],
    "image_id": s["image_id"],
    "bbox_x": s["object"]["x"],
    "bbox_y": s["object"]["y"],
    "bbox_w": s["object"]["w"],
    "bbox_h": s["object"]["h"],
    "bbox_size": s["object_size"],
    "object_name": s["object"]["name"],
    "attr_value": s["attr_value"],
    "y": s["y"]
} for s in attr_samples]
attr_samples_df = pd.DataFrame.from_dict(attr_samples_flat)

rel_samples_flat = []
for s in rel_samples:
    object0, object1 = s['object0'], s['object1']
    joined_bbox = {
        "y": min(object0['y'], object1['y']),
        "x": min(object0['x'], object1['x']),
        "h": max(object0['y'] + object0['h'], object1['y'] + object1['h']) - min(object0['y'], object1['y']),
        "w": max(object0['x'] + object0['w'], object1['x'] + object1['w']) - min(object0['x'], object1['x']),
    }
    rel_samples_flat.append({
        "question_id": s["question_id"],
        "image_id": s["image_id"],
        "bbox_x": joined_bbox["x"],
        "bbox_y": joined_bbox["y"],
        "bbox_w": joined_bbox["w"],
        "bbox_h": joined_bbox["h"],
        "bbox_size": compute_object_size(s["question"]["sceneGraph"], joined_bbox),
        "object0_name": s["object0"]["name"],
        "object1_name": s["object1"]["name"],
        "rel": s["rel"],
        "y": s["y"]
    })
rel_samples_df = pd.DataFrame.from_dict(rel_samples_flat)


In [6]:
import torch 
from torchvision.io import read_image, ImageReadMode
from torchvision.transforms.functional import crop, resize, pad
from math import tanh

def scaling(x, ceiling=3):
    return (1 - tanh(x * 2)) * ceiling

def get_scaled_bbox(entry, img_height, img_width, padding_scale_ceiling=0.5):
    padding_w = scaling(entry["bbox_w"] / img_width, padding_scale_ceiling) * entry["bbox_w"]
    padding_h = scaling(entry["bbox_h"] / img_height, padding_scale_ceiling) * entry["bbox_h"]

    return (
        int(max(entry["bbox_y"] - padding_h, 0)),
        int(max(entry["bbox_x"] - padding_w, 0)),
        int(min(entry["bbox_h"]+2*padding_h, img_height-max(entry["bbox_y"] - padding_h, 0))),
        int(min(entry["bbox_w"]+2*padding_w, img_width-max(entry["bbox_x"] - padding_w, 0)))
    )

class PromptDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, prompt_transform, img_size=224):
        self.df = df 
        self.prompt_transform = prompt_transform
        self.img_size = img_size

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        entry = self.df.iloc[idx]
        image = read_image(f"../data/images/{entry['image_id']}.jpg", ImageReadMode.RGB)

        # crop bounding box
        y,x,h,w = get_scaled_bbox(entry, image.shape[1], image.shape[2])
        orig_image = crop(image, entry["bbox_y"], entry["bbox_x"], entry["bbox_h"], entry["bbox_w"])
        image = crop(image, y, x, h, w)

        # resize and scale (maintain aspect ratio)
        if entry["bbox_h"] > entry["bbox_w"]:
            resize_dimensions = (self.img_size, 2*round((self.img_size*entry["bbox_w"]/entry["bbox_h"])/2)) 
        else:
            resize_dimensions = (2*round((self.img_size*entry["bbox_h"]/entry["bbox_w"])/2), self.img_size)
        image = resize(image, resize_dimensions)
        orig_image = resize(orig_image, (self.img_size, self.img_size))

        # pad the image to square dimensions
        image = pad(image, ((self.img_size - resize_dimensions[1])//2, (self.img_size - resize_dimensions[0])//2))

        return (image, self.prompt_transform(entry), entry['y'])


### Classes

In [9]:
from torch.utils.data import DataLoader

batch_size = 64
class_dataset = PromptDataset(class_samples_df, prompt_transform=lambda e: [
    f"a photo of a {e['class']}", 
    "a photo of an object"
])

In [134]:
class_dataloader = DataLoader(class_dataset, batch_size=batch_size, shuffle=True)
class_correct = 0

for batch in class_dataloader:
    image_inputs = image_processor(batch[0], return_tensors="pt", do_resize=False, do_center_crop=False).to("mps")
    text_inputs = tokenizer([*batch[1][0], *batch[1][1]], return_tensors="pt", padding=True).to("mps")
    num_items = batch[0].shape[0]

    result = model(**image_inputs, **text_inputs)
    scores = torch.stack([torch.diagonal(result["logits_per_image"][:,:num_items]), torch.diagonal(result["logits_per_image"][:,num_items:])])
    probs = torch.nn.functional.softmax(scores, dim=0)
    class_correct += sum((probs[0, :] > probs[1, :]) == batch[2].to("mps"))

print(f"Class Accuracy: {class_correct/class_dataset.__len__()*100}%")

Class Accuracy: 85.02111053466797%


### Attributes

In [16]:
attr_dataset = PromptDataset(attr_samples_df, prompt_transform=lambda e: [
    f"a photo of a {e['attr_value']} object", 
    f"a photo of a normal object"
])

In [17]:
attr_dataloader = DataLoader(attr_dataset, batch_size=batch_size, shuffle=True)
attr_correct = 0

for batch in attr_dataloader:
    image_inputs = image_processor(batch[0], return_tensors="pt", do_resize=False, do_center_crop=False).to("mps")
    text_inputs = tokenizer([*batch[1][0], *batch[1][1]], return_tensors="pt", padding=True).to("mps")
    num_items = batch[0].shape[0]
    
    result = model(**image_inputs, **text_inputs)
    scores = torch.stack([torch.diagonal(result["logits_per_image"][:,:num_items]), torch.diagonal(result["logits_per_image"][:,num_items:])])
    probs = torch.nn.functional.softmax(scores, dim=0)
    attr_correct += sum((probs[0, :] > probs[1, :]) == batch[2].to("mps"))

print(f"Attributes Accuracy: {attr_correct/attr_dataset.__len__()*100}%")

Attributes Accuracy: 50.086944580078125%


### Relations

In [14]:
rel_dataset = PromptDataset(rel_samples_df, prompt_transform=lambda e: [
    f"a photo of a {e['object0_name']} {e['rel']} a {e['object1_name']}", 
    f"a photo of a {e['object0_name']} and a {e['object1_name']}"
])

In [15]:
rel_dataloader = DataLoader(rel_dataset, batch_size=batch_size, shuffle=True)
rel_correct = 0

for batch in rel_dataloader:
    image_inputs = image_processor(batch[0], return_tensors="pt", do_resize=False, do_center_crop=False).to("mps")
    text_inputs = tokenizer([*batch[1][0], *batch[1][1]], return_tensors="pt", padding=True).to("mps")

    num_items = batch[0].shape[0]
    result = model(**image_inputs, **text_inputs)
    scores = torch.stack([torch.diagonal(result["logits_per_image"][:,:num_items]), torch.diagonal(result["logits_per_image"][:,num_items:])])
    probs = torch.nn.functional.softmax(scores, dim=0)
    rel_correct += sum((probs[0, :] > probs[1, :]) == batch[2].to("mps"))

print(f"Relations Accuracy: {rel_correct/rel_dataset.__len__()*100}%")

/Users/jan/Git/tuw-master-thesis/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Relations Accuracy: 55.75797653198242%
